# Libraries

In [1]:
%pip install -q -U datasets transformers accelerate sentencepiece

In [2]:
import os
import random
import numpy as np
import pandas as pd
import torch
import transformers
import unicodedata
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from pprint import pprint

# Config

In [3]:
seed = 42
lang = 'sun'

# hf_model_id = 'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20250112051102' # BEST
# hf_model_id = 'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20250113134514'
# hf_model_id = 'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20250112102813' # No. 63.3 - BEST disgust (Dev)
# hf_model_id = 'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20250107142404' # No. 42 - BEST fear (Dev)
hf_model_id = 'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20250107142404' # No. 42.1 - BEST surprise (Dev)
hf_tokenizer_id = hf_model_id
hf_data_id = 'alxxtexxr/SemEval2025-Task11-Dataset'
hf_data_config = 'raw_new_track_a_sun'

In [4]:
def set_seed(seed):
    # Set random seed for NumPy
    np.random.seed(seed)

    # Set random seed for Torch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensures deterministic results
    torch.backends.cudnn.benchmark = False  # Avoids non-deterministic algorithms

    # Set random seed for Transformers
    transformers.set_seed(seed)

    # Optionally set random seed for sklearn and Python's own random module
    random.seed(seed)

    # Set random seed for os
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seed set to: {seed}")

set_seed(seed)

Random seed set to: 42


In [5]:
datasets = load_dataset(hf_data_id, hf_data_config)
splits = [*datasets.keys()]
print("Splits:", splits)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Splits: ['train', 'dev', 'test']


In [6]:
split = 'test' # 'dev'
dev_df = pd.DataFrame(datasets[split])

cols = list(dev_df.columns)
emotion_cols_raw = [col for col in cols if col not in ['Unnamed: 0', 'id', 'text', 'emotion']]

# Add 'Neutral' column
dev_df['Neutral'] = None
emotion_cols = emotion_cols_raw + ['Neutral']

print("Data columns:", cols)
print("Emotions columns:", emotion_cols)

Data columns: ['id', 'text', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']
Emotions columns: ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'Neutral']


In [7]:
class2id = {class_:id for id, class_ in enumerate(emotion_cols)}
id2class = {id:class_ for class_, id in class2id.items()}

print("Class to ID:")
pprint(class2id, width=1)
print()
print("ID to Class:")
pprint(id2class, width=1)

Class to ID:
{'Neutral': 6,
 'anger': 0,
 'disgust': 1,
 'fear': 2,
 'joy': 3,
 'sadness': 4,
 'surprise': 5}

ID to Class:
{0: 'anger',
 1: 'disgust',
 2: 'fear',
 3: 'joy',
 4: 'sadness',
 5: 'surprise',
 6: 'Neutral'}


In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    hf_model_id, 
    num_labels=len(emotion_cols),
    id2label=id2class, 
    label2id=class2id,
    problem_type = "multi_label_classification",
)
print(model.classifier)

RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=7, bias=True)
)


In [9]:
tokenizer = AutoTokenizer.from_pretrained(hf_tokenizer_id)

In [10]:
threshold = 0.3

def sigmoid(x):
   return 1/(1 + np.exp(-x))

for i, row in dev_df.iterrows():
    text = row['text']
    inputs = tokenizer(text, return_tensors='pt').to(model.device)
    outputs = model(**inputs)
    logits = outputs.logits
    probs = sigmoid(logits.squeeze().detach().cpu().numpy()) # apply sigmoid + threshold
    labels_pred = (probs > threshold).astype(int) # threshold: 0.5
    
    dev_df.loc[i, emotion_cols] = labels_pred

In [11]:
save_df = dev_df[['id'] + emotion_cols_raw]
save_df.head()

,id,anger,disgust,fear,joy,sadness,surprise
0,sun_test_track_a_00001,0,0,0,1,0,0
1,sun_test_track_a_00002,0,0,0,1,0,0
2,sun_test_track_a_00003,1,1,0,1,1,1
3,sun_test_track_a_00004,0,0,0,1,1,0
4,sun_test_track_a_00005,0,0,0,1,0,0


In [14]:
save_path = 'pred_sun_42_1.csv'


save_df.to_csv(save_path, index=False)
print("Saved to:", save_path)

Saved to: pred_sun_42_1.csv
